In [40]:
import torch
from torch import nn
from torch.nn import functional as F

import d2l_ai as d2l

In [58]:
import math

In [41]:
queries = torch.normal(0, 1, (2, 12, 20))  # 批量大小为2，查询数为1，特征维度为20
keys = torch.ones((2, 10, 2))  # 键的数量为10，特征维度为2
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(2, 1, 1)  # 值的矩阵

In [42]:
queries.shape, queries

(torch.Size([2, 12, 20]),
 tensor([[[ 1.5363e+00,  5.9155e-01,  3.1239e-01,  1.7552e+00, -1.4724e+00,
            5.5754e-01, -5.0349e-01, -5.7015e-01, -1.5674e+00,  9.5721e-01,
           -3.8511e-01, -5.9959e-01, -2.5667e-01,  1.7399e+00,  9.0233e-01,
           -3.7771e-01, -1.3598e+00, -1.0398e-01,  3.0150e+00, -4.6976e-01],
          [ 1.1974e+00,  7.7358e-01,  4.1073e-01,  8.9882e-01, -1.4419e-01,
            1.7658e+00, -1.0064e+00,  9.1877e-01,  8.9527e-01, -5.7027e-01,
           -1.0110e+00, -1.0004e+00, -2.1531e-02,  1.3757e+00,  2.2733e+00,
            7.3325e-01, -2.1208e+00,  4.2902e-01, -2.5582e+00,  2.1686e-01],
          [ 5.4187e-01,  5.8805e-01, -5.1913e-01, -1.0505e+00,  4.1114e-01,
            8.7651e-01,  2.5390e-01, -4.5647e-01,  4.1540e-01, -6.8469e-02,
           -5.2871e-02,  6.7575e-01,  1.4666e+00,  1.3912e-01, -1.2540e+00,
            8.0966e-01,  7.5710e-01,  7.7924e-01,  5.9700e-01,  5.4004e-01],
          [ 2.2718e-01,  6.4448e-01,  1.8407e+00,  1.8630e+

In [43]:
valid_lens = torch.tensor([2, 6])  # 有效长度

In [44]:
key_size = 2
query_size = 20
num_hiddens = 8

In [45]:
W_k = nn.Linear(key_size, num_hiddens, bias=False)
W_q = nn.Linear(query_size, num_hiddens, bias=False)
w_v = nn.Linear(num_hiddens, 1, bias=False)

In [46]:
queries, keys = W_q(queries), W_k(keys)

In [47]:
queries.shape, keys.shape

(torch.Size([2, 12, 8]), torch.Size([2, 10, 8]))

In [48]:
features = queries.unsqueeze(2) + keys.unsqueeze(1)  # (batch_size, query_len, key_len, num_hiddens)

In [49]:
features.shape, features

(torch.Size([2, 12, 10, 8]),
 tensor([[[[ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072],
           [ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072],
           [ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072],
           ...,
           [ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072],
           [ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072],
           [ 0.1471, -0.4113, -1.0016,  ...,  1.3177, -1.3260, -0.6072]],
 
          [[ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174],
           [ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174],
           [ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174],
           ...,
           [ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174],
           [ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174],
           [ 1.9267, -0.3747, -0.7787,  ..., -0.4572, -1.0437, -1.0174]],
 
          [[ 0.8025, -0.2350, -0.5323,  ...,  0.4168, -1.

In [50]:
scores = w_v(features)
scores

tensor([[[[-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782],
          [-1.5782]],

         [[-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484],
          [-0.7484]],

         [[-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248],
          [-0.6248]],

         [[-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151],
          [-1.7151]],

         [[-0.6285],
          [-0.6285],
          [-0.6285],
          [-0.6285],
          [-0.6285],
          [-0.6285],
          [-0.6285],
     

In [51]:
scores.shape

torch.Size([2, 12, 10, 1])

In [52]:
scores = scores.squeeze(-1)

In [53]:
scores, scores.shape

(tensor([[[-1.5782, -1.5782, -1.5782, -1.5782, -1.5782, -1.5782, -1.5782,
           -1.5782, -1.5782, -1.5782],
          [-0.7484, -0.7484, -0.7484, -0.7484, -0.7484, -0.7484, -0.7484,
           -0.7484, -0.7484, -0.7484],
          [-0.6248, -0.6248, -0.6248, -0.6248, -0.6248, -0.6248, -0.6248,
           -0.6248, -0.6248, -0.6248],
          [-1.7151, -1.7151, -1.7151, -1.7151, -1.7151, -1.7151, -1.7151,
           -1.7151, -1.7151, -1.7151],
          [-0.6285, -0.6285, -0.6285, -0.6285, -0.6285, -0.6285, -0.6285,
           -0.6285, -0.6285, -0.6285],
          [-0.3240, -0.3240, -0.3240, -0.3240, -0.3240, -0.3240, -0.3240,
           -0.3240, -0.3240, -0.3240],
          [-1.1597, -1.1597, -1.1597, -1.1597, -1.1597, -1.1597, -1.1597,
           -1.1597, -1.1597, -1.1597],
          [-0.4600, -0.4600, -0.4600, -0.4600, -0.4600, -0.4600, -0.4600,
           -0.4600, -0.4600, -0.4600],
          [-0.7969, -0.7969, -0.7969, -0.7969, -0.7969, -0.7969, -0.7969,
           -0.7969, -0

In [54]:
valid_lens = torch.tensor([2, 6])

In [55]:
attention_weights = d2l.masked_softmax(scores, valid_lens)
attention_weights

tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000,

In [56]:
attention_weights.shape

torch.Size([2, 12, 10])

In [59]:
d = queries.shape[-1]  # 查询和键的维度

# 计算查询和键的点积，注意进行缩放
scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)

In [60]:
scores.shape, scores

(torch.Size([2, 12, 10]),
 tensor([[[ 0.4533,  0.4533,  0.4533,  0.4533,  0.4533,  0.4533,  0.4533,
            0.4533,  0.4533,  0.4533],
          [ 0.4496,  0.4496,  0.4496,  0.4496,  0.4496,  0.4496,  0.4496,
            0.4496,  0.4496,  0.4496],
          [ 0.0058,  0.0058,  0.0058,  0.0058,  0.0058,  0.0058,  0.0058,
            0.0058,  0.0058,  0.0058],
          [ 0.4706,  0.4706,  0.4706,  0.4706,  0.4706,  0.4706,  0.4706,
            0.4706,  0.4706,  0.4706],
          [-0.0829, -0.0829, -0.0829, -0.0829, -0.0829, -0.0829, -0.0829,
           -0.0829, -0.0829, -0.0829],
          [-0.2745, -0.2745, -0.2745, -0.2745, -0.2745, -0.2745, -0.2745,
           -0.2745, -0.2745, -0.2745],
          [ 0.5166,  0.5166,  0.5166,  0.5166,  0.5166,  0.5166,  0.5166,
            0.5166,  0.5166,  0.5166],
          [ 0.2556,  0.2556,  0.2556,  0.2556,  0.2556,  0.2556,  0.2556,
            0.2556,  0.2556,  0.2556],
          [-0.2755, -0.2755, -0.2755, -0.2755, -0.2755, -0.2755, -0.27

In [61]:
attention_weights = d2l.masked_softmax(scores, valid_lens)

In [62]:
attention_weights

tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000,

In [63]:
attention_weights.shape

torch.Size([2, 12, 10])

In [ ]:
attention_weights.reshape((1, 1, 2, 10))